In [45]:
import torch
from d2l import torch as d2l
from torch import nn
from torch.nn import functional as F

In [49]:
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(20, 1)
)

X = torch.rand(2, 20)
X, net(X)

(tensor([[0.6937, 0.8161, 0.3049, 0.0464, 0.9902, 0.0281, 0.3335, 0.3288, 0.7410,
          0.3239, 0.3877, 0.4888, 0.7052, 0.8520, 0.6238, 0.6250, 0.3131, 0.6907,
          0.5637, 0.2817],
         [0.6776, 0.3193, 0.7141, 0.7756, 0.1115, 0.6014, 0.4594, 0.3991, 0.8274,
          0.7802, 0.9634, 0.5700, 0.8236, 0.4451, 0.2913, 0.1873, 0.1509, 0.3473,
          0.9625, 0.4917]]),
 tensor([[-0.2672],
         [-0.2322]], grad_fn=<AddmmBackward0>))

In [56]:
class net_hand(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.out = nn.Linear(20, 1)


    def forward(self, X):
        return self.out(X)

In [73]:
net = nn.Sequential(
    nn.Linear(20, 1),
    nn.ReLU(),
    nn.Linear(1,20),
    nn.Linear(20,1)
)
net(X)
net[0].state_dict()

OrderedDict([('weight',
              tensor([[-0.1396,  0.0577, -0.1841, -0.0773,  0.1662, -0.1372,  0.1696, -0.0634,
                       -0.0906,  0.1315, -0.1623, -0.1756, -0.1449, -0.1290,  0.2217,  0.0742,
                        0.1669,  0.0515, -0.0890, -0.0701]])),
             ('bias', tensor([0.1077]))])

In [68]:
net

Sequential(
  (0): Linear(in_features=20, out_features=1, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1, out_features=20, bias=True)
  (3): Linear(in_features=20, out_features=5, bias=True)
)

In [87]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

# for i in range(4):
#     print(f'第{i}层')
#     print(net[i].state_dict())

('weight', torch.Size([1, 20])) ('bias', torch.Size([1]))
('0.weight', torch.Size([1, 20])) ('0.bias', torch.Size([1])) ('2.weight', torch.Size([20, 1])) ('2.bias', torch.Size([20])) ('3.weight', torch.Size([1, 20])) ('3.bias', torch.Size([1]))


In [91]:
def init(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 2)
        nn.init.zeros_(m.bias)

# net[0].apply(init)
# net[0].weight.mean(), net[0].bias
net[0].weight.shape
net[0].apply(init)
net[0].state_dict()

OrderedDict([('weight',
              tensor([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
                       2., 2.]])),
             ('bias', tensor([0.]))])

模型的存储与加载

In [92]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [93]:
torch.save(net.state_dict(), 'mlp.params')

In [94]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval() # 设置 dropout 和 batch normalization 层为评估模式。如果不这么做，可能导致 模型推断结果不一致。

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [95]:
Y_new = clone(X)
Y == Y_new

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])